## Summarizing a text using an LLM

As an LLM "understands" a language, it can be suited for tasks like translation or summarization.

In this Notebook, we are going to use our LLM to summarize some texts, especially claims examples.

### Requirements and Imports

If you have selected the right workbench image to launch as per the Lab's instructions, you should already have all the needed libraries. If not uncomment the first line in the next cell to install all the right packages.

In [1]:
# Uncomment the following line only if you have not selected the right workbench image, or are using this notebook outside of the workshop environment.
# !pip install --no-cache-dir --no-dependencies --disable-pip-version-check -r requirements.txt

import json
import os
import httpx
from os import listdir
from os.path import isfile, join

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

### Langchain pipeline

Again, we are going to use Langchain to define our summarization pipeline.

In [2]:
# LLM Inference Server URL
inference_server_url = "http://llama3-2-3b-predictor.llama-serving.svc.cluster.local:8080/v1"
inference_server_model_name = "llama3-2-3b"


# LLM definition
llm = ChatOpenAI(
    model=inference_server_model_name,
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key="EMPTY",  # if you prefer to pass api key in directly instaed of using env vars
    base_url=inference_server_url,
    http_client=httpx.Client(verify=False)    # Because we are using an internal API endpoint (service) we need to disable SSL certificate checking.
)

The **template** we will use is now formatted for this specific summarization task.

In [3]:
template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        """You are a helpful, respectful and honest assistant.
        Always assist with care, respect, and truth. Respond with utmost utility yet securely.
        Avoid harmful, unethical, prejudiced, or negative content. Ensure replies promote fairness and positivity.
        I will give you a text that you must summarize as best as you can.
        """),
    HumanMessagePromptTemplate.from_template(
        """### TEXT:
        {input}
        ### SUMMARY:"""
    )
])

We are now ready to query the model!

In the `claims` folder we have JSON files with examples of claims that could be received. We are going to read those files, display them, then the summary that the LLM made.

In [4]:
# Read the claims and populate a dictionary
claims_path = 'claims'
onlyfiles = [f for f in listdir(claims_path) if isfile(join(claims_path, f))]

claims = {}

for filename in onlyfiles:
    # Opening JSON file
    with open(os.path.join(claims_path, filename), 'r') as file:
        data = json.load(file)
    claims[filename] = data

In [5]:
for filename in onlyfiles:
    print("***************************")
    print(f"* Claim: {filename}")
    print("***************************")
    print("===== ORIGINAL CONTENT =====\n\n")
    print(f"Subject: {claims[filename]['subject']}\nContent:\n{claims[filename]['content']}\n\n")
    print('===== SUMMARY =====\n\n')
    summary_input = f"Subject: {claims[filename]['subject']}\nContent:\n{claims[filename]['content']}"
    prompt = template.invoke({"input": summary_input})
    resp = llm.invoke(input=prompt)
    print(resp.content)  # Access the text
    print("\n\n===== END OF SUMMARY =====\n\n")


***************************
* Claim: claim1.json
***************************
===== ORIGINAL CONTENT =====


Subject: Claim for Recent Car Accident - Policy Number: AC-987654321
Content:
Dear Parasol Insurance,

I hope this email finds you well. My name is Sarah Turner, and I am writing to file a claim for a recent car accident that occurred on January 2nd, 2024, at approximately 3:30 PM. My policy number is AC-987654321.

The accident took place at the intersection of Birch Street and Willow Avenue in the city of Evergreen. I was driving my vehicle, a black Toyota Camry with license plate number DEF-456, heading south on Birch Street. At the intersection, the traffic signal was green, and I proceeded through the intersection.

At the same time, another vehicle, a blue Chevrolet Traverse with license plate number GHI-789, was traveling west on Willow Avenue. Unfortunately, the driver failed to stop at the red traffic signal, resulting in a collision with the front passenger side of my v

In [6]:
print("Summarisation complete.")

Summarisation complete.


---
# End of activity
Return to the Roadshow instructions for next steps.